In [52]:
import pandas as pd
import os
import gzip
import pickle
import openai
import re
import copy
from tqdm import tqdm
from torch import save,load
import time

concatenated_threads = pd.read_csv('../2_train_set_creation/concatenated_threads.csv')

id = 0

hardware_instances = []
hardware_instances_inc_threads = []

# for i in range(1, 14):
for (i,file) in enumerate(os.listdir('output_chunks/')):
    i = i+1
    # print(i)
    texts = load(f'output_chunks/outputs_chunk_{i}.pt')
    for text in texts:
        output = text.split("Hardware names:")[1]
        try:
            date = text.split("Date:")[1].split("Topic:")[0].strip()
        except:
            print("something went wrong")
            # print(text)
            id+=1
            continue

        # print("id: ", id)
        # print(date)
        # print(concatenated_threads.iloc[id]['date'])
        # print(output)

        # date = concatenated_threads.iloc[id]['date']

        

        # figure out how many posts in the thread were actually given to the model
        # we need a regex to match stuff like "### Reply 2:"
        reply_matches = re.findall(r"### Reply \d+:", text)
        posts_count = len(reply_matches) + 1 # we add 1 because the first post is not matched by the regex
        dates = concatenated_threads.iloc[id]['dates'].split("<sep>")

        if len(dates) > 50:
            # check if last date is at least 200 days after the first date
            # dates are in format 2013-04-20 01:49:43
            first = time.mktime(time.strptime(dates[0], "%Y-%m-%d %H:%M:%S"))
            last = time.mktime(time.strptime(dates[posts_count-1], "%Y-%m-%d %H:%M:%S"))
            if last - first > 200*24*60*60:
                print("skipping id: ", id)
                # print(reply_matches)
                # print(posts_count)
                # print(dates)
                # print("\n")

                id+=1
                continue

        date = dates[posts_count-1]

        sp = output.split("Hardware ownership:")
        if len(sp) != 2:
            print("something went wrong2")
            # print(text)
            id+=1
            continue
        names = sp[0].strip()
        ownerships = sp[1].strip()
        for (name, ownership) in zip(names.split(","), ownerships.split(",")):
            n = name.strip().lower()
            o = ownership.strip()
            if o == "True":
                hardware_instances.append((date, n))

        for (name, ownership) in zip(names.split(","), ownerships.split(",")):
            n = name.strip().lower()
            o = ownership.strip()
            if o == "True":
                hardware_instances_inc_threads.append((date, text))
                break
        
        id+=1
        # break

    # break
        


something went wrong2
something went wrong
something went wrong2
something went wrong2
something went wrong
something went wrong
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong
something went wrong2
something went wrong2
something went wrong2
something went wrong
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
skipping id:  8614
something went wrong2
something went wrong2
something went wrong2
something went wrong
something went wrong2
something went wrong
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
something went wrong2
so

In [53]:
df = pd.DataFrame(hardware_instances, columns=['date', 'hardware'])
df = df.sort_values(by=['date'])
hardware_instances

[('2013-03-26 00:19:57', '3-module one'),
 ('2013-03-26 05:58:09', 'avalon'),
 ('2013-03-26 15:21:55', 'video cards'),
 ('2013-03-26 19:32:53', 'avalon batch 3'),
 ('2013-03-26 19:32:53', 'fpga bitcoin mining (3 ztex quadra)'),
 ('2013-03-26 19:32:53', 'gpu litecoin mining rig with 4 hd 7850'),
 ('2013-03-27 22:02:32', 'modules'),
 ('2013-03-27 22:02:32', 'miners'),
 ('2013-03-28 09:38:41', 'avalon batch 3'),
 ('2013-03-28 09:38:41', 'gpu farm'),
 ('2013-03-27 21:50:02', 'bfl orders'),
 ('2013-03-28 21:18:04', 'corporate datacenter server room'),
 ('2013-03-28 21:18:04', '100mbit fibreline'),
 ('2013-03-28 21:18:04', 'dual 10kva ups'),
 ('2013-03-30 06:08:10', 'modules'),
 ('2013-03-30 15:00:21', 'batch 2 units'),
 ('2013-04-18 16:49:52', 'pcbs'),
 ('2013-04-20 08:38:33', 'qfn-28'),
 ('2013-04-20 08:38:33', 'eagle'),
 ('2013-04-20 08:38:33', 'orcad'),
 ('2013-04-20 08:38:33', 'geda - gschem and pcb'),
 ('2013-04-23 04:55:23', 'avalon asic chips'),
 ('2013-04-23 16:49:25', 'fpga'),
 ('2

In [54]:
df.to_csv('hardware_instances.csv', index=False)

In [55]:
df = pd.DataFrame(hardware_instances_inc_threads, columns=['date', 'thread'])
df = df.sort_values(by=['date'])
hardware_instances_inc_threads

[('2013-03-26 00:19:57',
  'User:\nIn the given Bitcoin forum thread, pay close attention to the language used when mentioning hardware pieces. Look for explicit statements indicating ownership or hypothetical discussions.\n\n```thread\nDate: 2013-03\nTopic:  CANCELLED [Group Buy in China]  Avalon batch #3\n### Original post:\nWhy was it cancelled?\n\n### Reply 1:\nBatch #3 already sold out.\n\n### Reply 2:\nSeems still available, just checked !?\n\n### Reply 3:\n153 in stock\n\n### Reply 4:\nWhere do you look for that number ?\n\n### Reply 5:\nWill tell you for 2 bitcoins ). Upd: 152\n\n### Reply 6:\ngo on <link> try to buy 999 units. it will fail and tell you how many units are actually in stock.\n\n### Reply 7:\nI don\'t have any left, spent them all on a 3-module one, this morning when they opened the sale\n\n\n```\n\n\n\nReply with the hardware names, all in the same line, separated by commas. Then, on a new line, list "True" or "False" for each piece of hardware to indicate owner

In [56]:
df.to_csv('hardware_instances_inc_threads.csv', index=False)